In [89]:
import numpy as np

In [90]:

def load_data(fname):
    with open(fname,'r') as f:
        utt_ID = [line.split(None, 1)[0] for line in f]
    with open(fname,'r') as f:
        y = [line.split('\t', 2)[1] for line in f]
    with open(fname,'r') as f:
        x_train_org =[line.strip().split('\t', 2)[2] for line in f]
        x_train = [x.split(';') for x in x_train_org]
    return utt_ID,y,x_train 
    

In [91]:
utt_ID_train,y_train,x_train_org= load_data("utterances1.train")
x_train_org[0]
len(x_train_org)

196502

In [92]:
# Padding the input data
input_length = 10 # average length 

def pad(ut):
    return ut[:10]+['' for i in range(input_length-len(ut))]


In [93]:
x_train=[None]*len(x_train_org)
for i in range(len(x_train_org)):
    res = []
    for list in x_train_org[i]:
        res.append(str(list).lower().split())
    x_train[i]=res

In [94]:
#x_train_proc = processX(x_train_org)

x_train_pad= np.array([[ pad(ut) for ut in dialog] for dialog in x_train])

In [42]:
uniqueWords = []
for i in range(len(x_train_org)):
        for j in range(len(x_train_pad[i])):
             for k in x_train[i][j]:
                    if not k.lower() in uniqueWords:
                        uniqueWords.append(k.lower())
print(len(uniqueWords))

92


In [95]:
uniqueWords=np.unique([word for dialog in (x_train_pad) for ut in dialog for word in ut])
print(len(uniqueWords))

16739


In [96]:
i=1
vocab ={}
for x in uniqueWords:
    vocab.update({x:i})
    i=i+1
#for i in uniqueWords:
#    print(i.lower())
#INDEX_FROM = 2 

# Dict {word:id}
word_to_id = vocab#{x:vocab[x]+INDEX_FROM for x in vocab if vocab[x]<=2000}
#word_to_id["<START>"] = 1
#word_to_id["<UNK>"] = 2


# Dict {id:word}
id_to_word = {word_to_id[x]:x for x in word_to_id}

In [45]:
def get_W(word_vecs, k=300):
    """
    Get word matrix. W[i] is the vector for word indexed by i
    """
    vocab_size = len(word_vecs)
    word_idx_map = dict()
    W = np.zeros(shape=(vocab_size + 1, k), dtype='float32')
    W[0] = np.zeros(k, dtype='float32')
    i = 1
    for word in word_vecs:
        W[i] = word_vecs[word]
        word_idx_map[word] = i
        i += 1
    return W, word_idx_map


def load_bin_vec(fname, vocab):
    """
    Loads 300x1 word vecs from Google (Mikolov) word2vec
    """
    word_vecs = {}
    with open(fname, "rb") as f:
        header = f.readline()
        # ~ print(header)
        vocab_size, layer1_size = map(int, header.split())
        binary_len = np.dtype('float32').itemsize * layer1_size
        # print(vocab_size)
        for line in range(vocab_size):
            # print(line)
            word = []
            while True:
                ch = f.read(1).decode('iso-8859-1')
                if ch == ' ':
                    word = ''.join(word)
                    break
                if ch != '\n':
                    word.append(ch)
            # print(word)
            if word in vocab:
                # print(word)
                word_vecs[word] = np.frombuffer(f.read(binary_len), dtype='float32')
            else:
                f.read(binary_len)

    return word_vecs

def add_unknown_words(word_vecs, vocab, min_df=1, k=300):
    """
    For words that occur in at least min_df documents, create a separate word vector.    
    0.25 is chosen so the unknown vectors have (approximately) same variance as pre-trained ones
    """
    for word in vocab:
        if word not in word_vecs and vocab[word] >= min_df:
            word_vecs[word] = np.random.uniform(-0.25, 0.25, k)
            

In [97]:
vocab_list = np.array([id_to_word[x] for x in range(1,len(uniqueWords)+1)])

In [98]:
import time
start = time.time()
w2v_file = "GoogleNews-vectors-negative300.bin"
w2v = load_bin_vec(w2v_file, word_to_id)
print("num words found: %d" % len(w2v))
add_unknown_words(w2v, word_to_id, k=300)
W, word_idx_map = get_W(w2v, k=300)

print("W shape: %s" % str(W.shape))

print("%d seconds to get the embeddings" % (time.time()-start))

num words found: 14246
W shape: (16740, 300)
68 seconds to get the embeddings


In [48]:
x_embedded_train = np.empty(len(x_train_org),dtype=object)

for id_i, i in enumerate(x_train):
    x_embedded_train_uttr = np.empty(len(x_train[0]),dtype=object)
    for j in range(len(i)):
        if(len(i[j])==0):
            temp=np.zeros([1,300],dtype=float)
            x_embedded_train_uttr[j]=np.array(temp)
        else:
            x_embedded_train_uttr_k = np.zeros([len(i[j]),300],dtype=float)
            for k in range(len(i[j])):
                x_embedded_train_uttr_k[k]= w2v[i[j][k]]
            x_embedded_train_uttr[j]=np.array(x_embedded_train_uttr_k)
    x_embedded_train[id_i] = np.array(x_embedded_train_uttr)
                

KeyError: 'air'

In [49]:
def embedding(x_proc):
    x_embedded = np.array([[[W[word_idx_map[word]] for word in ut] for ut in dialog] for dialog in x_proc])
    x_flatten = np.array([[ ut.flatten() for ut in dialog] for dialog in x_embedded])
    return x_flatten


In [52]:
x_em= embedding(x_train_pad)
x_em.shape

(10, 4, 3000)

In [60]:
#Labels
labels = np.array(['%', '%--', '2', 'aa', 'aap', 'ar', 'b', 'ba', 'bc', 'bd', 'bh', 'bk', 'br', 'bs', 'cc', 'co', 'd', 'fa', 'ft', 'g', 'h', 'no', 'qh', 'qo', 'qrr', 'qw', 'qy', 's', 't1', 't3','x']).tolist()

id_to_label = dict()
for i in range(len(labels)):
    id_to_label[labels[i]] = i

def labelToIndex(y):
    y_encoded = [id_to_label[l] for l in y]
    #print(y_encoded)
    return np.array(y_encoded)

def indexToLabel(pred_y):
    y_labels = [labels[p] for p in pred_y]
    #print(y_labels)
    return y_labels

In [77]:
print("Original train: ", y_train)
y_tr = labelToIndex(y_train)
print("Encoded train: ", y_tr)
from keras.utils import np_utils

y_train_cat = np_utils.to_categorical(y_tr)
num_classes=np.unique(y_tr).size

Original train:  ['bc', 'qw', 'h', 's', 'qo', 's', 's', 's', 'b', 'qy']
Encoded train:  [ 8 25 20 27 23 27 27 27  6 26]


In [86]:
num_classes = y_train_cat.shape[1]
print(num_classes)

28


In [102]:
from keras.models import Model
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense,Bidirectional
#from keras.preprocessing import sequence
#input_length=4
model=Sequential()
time_steps=x_em.shape[1]
features=x_em.shape[2]
 
model.add(Bidirectional(LSTM(units= 10,  activation='tanh'),input_shape=(time_steps,features)))
#model.add(Dense(10, activation='relu'))
model.add(Dense(num_classes, activation='tanh'))
print(model.summary())
# plot_model(model, to_file='Sent_FF.png', show_shapes=False, show_layer_names=True, rankdir='TB')

# loss function = binary_crossentropy
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_4 (Bidirection (None, 20)                240880    
_________________________________________________________________
dense_13 (Dense)             (None, 28)                588       
Total params: 241,468
Trainable params: 241,468
Non-trainable params: 0
_________________________________________________________________
None


In [103]:
#model.fit(x_embedded_train, y_train, validation_data=(x_embedded_train, y_train), epochs=2, batch_size=128, verbose=2)
#model.fit(x_em, y_train_cat, validation_data=(x_em, y_train_cat), epochs=2, batch_size=128, verbose=2)
model.fit(x_em, y_train_cat,  epochs=2, batch_size=128, verbose=2)

Epoch 1/2
 - 4s - loss: 0.3273 - acc: 0.9643
Epoch 2/2
 - 0s - loss: 0.2100 - acc: 0.9643


In [15]:
lengths = []
for i in x_train:
    for j in i:
        lengths.append(len(j))

print("max %d" % max(lengths))
print("mean %d" % np.mean(lengths))
max_length=max(lengths)
mean_length=np.int(np.mean(lengths))

max 66
mean 15


In [14]:
x[0]

'i'